In [12]:
datas = [{'Entity': 'Protein', 'Copies': 1, 'Content': 'QPHIGNYRLLKTLGKGNFAVVKLARHILTGKEVAIKIIDKTQLNSSSLQKIFREVRIMKVLNHPNIIKLFEVIETEKTLYLVTEYAKNGEVFDYLVAHGRMKEKEARAKFRQIVSAVQYCHQKFIVHRDLKAENLLLDADMNIKIADFGFGNEFTFGNKLDTFCGSPPYAAPELFQGKKYDGPEVDVWSLGVILYTLVSGSLPFDGQNLKELRERVLRGKYRIPFYMSTDCENLLKKFLILNPSKRGTLEQIMKDRWMNVGHEDDELKPYVEPLPDYKDPRRTELMVSMGYTREEIQDSLVGQRYNEVMATYLLLGY'}, {'Entity': 'Ligand', 'Copies': 1, 'Content': 'CN1CC[C@@H](COc2cnc(C#N)cc2-c2ccn3nc(NC(=O)C4CC4)cc3c2)C1'}]

In [16]:
print(datas[1])
datas[0]

{'Entity': 'Ligand', 'Copies': 1, 'Content': 'CN1CC[C@@H](COc2cnc(C#N)cc2-c2ccn3nc(NC(=O)C4CC4)cc3c2)C1'}


{'Entity': 'Protein',
 'Copies': 1,
 'Content': 'QPHIGNYRLLKTLGKGNFAVVKLARHILTGKEVAIKIIDKTQLNSSSLQKIFREVRIMKVLNHPNIIKLFEVIETEKTLYLVTEYAKNGEVFDYLVAHGRMKEKEARAKFRQIVSAVQYCHQKFIVHRDLKAENLLLDADMNIKIADFGFGNEFTFGNKLDTFCGSPPYAAPELFQGKKYDGPEVDVWSLGVILYTLVSGSLPFDGQNLKELRERVLRGKYRIPFYMSTDCENLLKKFLILNPSKRGTLEQIMKDRWMNVGHEDDELKPYVEPLPDYKDPRRTELMVSMGYTREEIQDSLVGQRYNEVMATYLLLGY'}

In [17]:
import os
import sys
import json
import shutil
import pandas as pd
from datetime import datetime
import subprocess
from concurrent.futures import ThreadPoolExecutor

def boltzInputDataGenerate(pdb_id, datas, folder_path):
    boltz_input_file = f"{folder_path}/{pdb_id}.fasta"
    # 알파벳 리스트 생성
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    alphabet_idx = 0  # 알파벳 인덱스 초기화

    with open(boltz_input_file, "w") as fasta:
        for data in datas:
            for _ in range(data['Copies']):  # Copies 수만큼 반복
                if alphabet_idx >= len(alphabet):  # 알파벳이 다 쓰이면 종료
                    break
                current_alphabet = alphabet[alphabet_idx]  # 현재 알파벳 선택
                alphabet_idx += 1  # 다음 알파벳으로 이동

                if data['Entity'] == 'Protein':
                    fasta.write(f">{current_alphabet}|protein|\n")
                    fasta.write(f"{data['Content']}\n")
                elif data['Entity'] == 'Ligand':
                    fasta.write(f">{current_alphabet}|smiles|\n")
                    fasta.write(f"{data['Content']}\n")
        
    return boltz_input_file

def createInputFolder(job):
    current_time = datetime.now().strftime("%Y%m%d_%Hh_%Mm_%Ss")
    input_file_path = os.path.join(f"/opt/git_tools/_development/DW_protein_AI_web/dw_ai_protein_web/datas/inputs/{job}/{current_time}")
    if not os.path.exists(input_file_path):
        os.makedirs(input_file_path)
    origin_output_dir = f"/opt/git_tools/_development/DW_protein_AI_web/dw_ai_protein_web/datas/outputs/{job}/{current_time}_result"
    return input_file_path, origin_output_dir

def runBoltz(input_path, output_path):
    if not input_path or not output_path:
        print("Error: Both input and output paths must be provided.")
        sys.exit(1)
    
    recycling_steps = 3
    sampling_steps = 200
    num_workers = 4
    
    # Boltz 실행 명령어 구성
    run = [
        "/opt/anaconda3/envs/boltz/bin/boltz",
        "predict",
        f"{input_path}",
        "--out_dir", f"{output_path}/",
        "--cache", "/opt/git_tools/boltz/model/",
        "--recycling_steps", str(recycling_steps),
        "--sampling_steps", str(sampling_steps),
        "--diffusion_samples", "5",
        "--num_workers", str(num_workers),
        "--use_msa_server"
    ]
     # 계산 시작 시간 기록
    start_time = datetime.now()
    print(f"Boltz 계산 중... 잠시만 기다려 주세요. (시작 시간: {start_time})")

    # Boltz 실행
    result_boltz = subprocess.run(run, capture_output=True, text=True)

    # 계산 종료 시간 기록
    end_time = datetime.now()

    if result_boltz.returncode != 0:
        print("Boltz 실행 중 오류가 발생했습니다.")
        print(f"오류 메시지: {result_boltz.stderr}")
    else:
        elapsed_time = end_time - start_time
        print(f"결과는 {output_path}에서 저장되었습니다!")
        print(f"결과 파일은 boltz_results_[input 파일 이름]/predictions/[input 파일 이름] 내에 .cif 형태로 저장되었습니다!")
        print(f"총 소요 시간: {elapsed_time}")

        # 결과 디렉토리에서 스코어 추출 및 CSV 저장
        # 입력 파일 이름에서 확장자 제거
        input_basename = os.path.splitext(os.path.basename(input_path))[0]
        predictions_folder = os.path.join(output_path, f"boltz_results_{input_basename}/predictions/{input_basename}")
        
        if not os.path.exists(predictions_folder):
            print(f"예상된 결과 폴더가 없습니다: {predictions_folder}")
        else:
            score_files = [f for f in os.listdir(predictions_folder) if f.startswith("confidence_") and f.endswith(".json")]
            
            if not score_files:
                print("스코어 파일이 발견되지 않았습니다.")
            else:
                # 주요 스코어 추출
                data = []
                for score_file in score_files:
                    score_path = os.path.join(predictions_folder, score_file)
                    with open(score_path, "r") as f:
                        json_data = json.load(f)
                    
                    # 주요 스코어 추출
                    confidence_score = json_data.get("confidence_score")
                    ptm = json_data.get("ptm")
                    iptm = json_data.get("iptm")
                    ligand_iptm = json_data.get("ligand_iptm")
                    complex_plddt = json_data.get("complex_plddt")
                    complex_iplddt = json_data.get("complex_iplddt")
                    
                    # 데이터 추가
                    data.append({
                        "File": score_file,
                        "Confidence Score": confidence_score,
                        "PTM": ptm,
                        "IPTM": iptm,
                        "Ligand IPTM": ligand_iptm,
                        "Complex pLDDT": complex_plddt,
                        "Complex ipLDDT": complex_iplddt,
                    })

                # DataFrame으로 변환 및 반올림
                df = pd.DataFrame(data)
                df = df.round(3)  # 소수점 세 자리로 반올림

                # CSV 저장
                csv_path = os.path.join(predictions_folder, "scores_summary.csv")
                df.to_csv(csv_path, index=False)
                print(f"스코어 요약이 CSV 파일로 저장되었습니다: {csv_path}")
    
    return predictions_folder

def makeZip(outfoler):
    # 압축 파일 저장 경로와 이름 (zip 형식)
    output_zip_file = f"{outfoler}.zip"
    shutil.make_archive(
        base_name=output_zip_file.replace(".zip", ""),  # 확장자 제외한 이름
        format="zip",  # 압축 형식
        root_dir=outfoler  # 압축 대상 폴더
    )
    print(f"압축 완료: {output_zip_file}")
    # 압축 대상 폴더 삭제
    if os.path.exists(outfoler):
        shutil.rmtree(outfoler)
        print(f"폴더 삭제 완료: {outfoler}")
    else:
        print(f"폴더가 존재하지 않습니다: {outfoler}")
    
    return output_zip_file

if __name__ == "__main__":
    job = "boltz"
    input_file_path, origin_output_dir = createInputFolder(job)
    boltz_input_file = boltzInputDataGenerate(job, datas, input_file_path)
    
    # 병렬 처리
    with ThreadPoolExecutor(max_workers=4) as executor:
        result_boltz = executor.submit(runBoltz, boltz_input_file, origin_output_dir)
        # makeZip(result_boltz.result().args[3])


Boltz 계산 중... 잠시만 기다려 주세요. (시작 시간: 2024-12-19 17:41:13.806569)
결과는 /opt/git_tools/_development/DW_protein_AI_web/dw_ai_protein_web/datas/outputs/boltz/20241219_17h_41m_13s_result에서 저장되었습니다!
결과 파일은 boltz_results_[input 파일 이름]/predictions/[input 파일 이름] 내에 .cif 형태로 저장되었습니다!
총 소요 시간: 0:01:35.569205
스코어 요약이 CSV 파일로 저장되었습니다: /opt/git_tools/_development/DW_protein_AI_web/dw_ai_protein_web/datas/outputs/boltz/20241219_17h_41m_13s_result/boltz_results_boltz/predictions/boltz/scores_summary.csv


In [19]:
result_boltz.result()
"/".join(result_boltz.result().split('/')[:-3])

'/opt/git_tools/_development/DW_protein_AI_web/dw_ai_protein_web/datas/outputs/boltz/20241219_17h_41m_13s_result'

In [10]:
path_test = "/opt/git_tools/_development/DW_protein_AI_web/dw_ai_protein_web/datas/outputs/boltz/20241219_16h_00m_57s_result/boltz_results_boltz/predictions/boltz/"
"/".join(result_boltz.result().split('/')[:-3])

'/opt/git_tools/_development/DW_protein_AI_web/dw_ai_protein_web/datas/outputs/boltz/20241219_16h_00m_57s_result/boltz_results_boltz'

In [25]:
pdb_id = "boltz_test"
boltz_input_dir = boltzInputDataGenerate(pdb_id, datas, folder_path="")

In [26]:
boltz_input_dir

'/opt/git_tools/_development/DW_protein_AI_web/test'

___

In [ ]:
model_seeds = [1]
# JSON 생성
json_data = {
    "name": pdb_id,
    "sequences": [
        {
            "protein": {
                "id": "P",
                "sequence": protein_sequence
            }
        },
        {
            "ligand": {
                "id": "L",
                "smiles": ligand_smiles
            }
        }
    ],
    "modelSeeds": model_seeds,
    "dialect": "alphafold3",
    "version": 1
}

# JSON 문자열로 변환
json_string = json.dumps(json_data, indent=4)

In [ ]:
import json

def alphafoldInputDataGenerateToJSON(pdb_id, datas, folder_path):
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    alphabet_idx = 0

    sequences = []  # JSON 데이터를 위한 시퀀스 리스트

    for data in datas:
        for _ in range(data['Copies']):
            if alphabet_idx >= len(alphabet):
                break
            current_alphabet = alphabet[alphabet_idx]
            alphabet_idx += 1

            if data['Entity'] == 'Protein':
                sequences.append({
                    "protein": {
                        "id": current_alphabet,
                        "sequence": data['Content']
                    }
                })
            elif data['Entity'] == 'Ligand':
                sequences.append({
                    "ligand": {
                        "id": current_alphabet,
                        "smiles": data['Content']
                    }
                })

    # JSON 생성
    json_data = {
        "name": pdb_id,
        "sequences": sequences,
        "modelSeeds": [1],
        "dialect": "alphafold3",
        "version": 1
    }

    # JSON 파일 저장
    json_file = f"{folder_path}/{pdb_id}.json"
    with open(json_file, "w") as json_out:
        json.dump(json_data, json_out, indent=4)

    return json_file

# 사용 예시
pdb_id = "example_pdb"
datas = [
    {"Entity": "Protein", "Content": "MVLSPADKTNVKAAW", "Copies": 2},
    {"Entity": "Ligand", "Content": "C1=CC=CC=C1", "Copies": 3}
]
folder_path = "/opt/git_tools/_development/DW_protein_AI_web/test"

json_file = boltzInputDataGenerateToJSON(pdb_id, datas, folder_path)
print(f"JSON file saved to: {json_file}")

JSON file saved to: /opt/git_tools/_development/DW_protein_AI_web/test/example_pdb.json


In [10]:
import os

folder_path = "/opt/git_tools/_development/DW_protein_AI_web/dw_ai_protein_web/datas/outputs/alphafold3/20250102_07h_03m_51s_result"  # PyMOL 파일이 저장된 경로

files = []
for root, dirs, filenames in os.walk(folder_path):
    for f in filenames:
        if f.endswith(('.pdb', '.cif')):
            # 상대 경로로 추가
            files.append(os.path.relpath(os.path.join(root, f), start=folder_path))

files

# '/'가 없는 문자열만 추출
change_file_name = [f for f in files if '/' not in f])

change_file_name

PermissionError: [Errno 13] Permission denied: '/opt/git_tools/_development/DW_protein_AI_web/dw_ai_protein_web/datas/outputs/alphafold3/20250102_07h_03m_51s_result/20250102_07h_03m_51s_result_model.cif' -> '/opt/git_tools/_development/DW_protein_AI_web/dw_ai_protein_web/datas/outputs/alphafold3/20250102_07h_03m_51s_result/renamed_20250102_07h_03m_51s_result_model.cif'

In [5]:
files = [f for f in os.listdir(folder_path) if f.endswith(('.pdb', '.cif'))]
files

['20250102_07h_03m_51s_result_model.cif']